In [35]:
import numpy as np
import pandas as pd

In [36]:
train = pd.read_csv('Restaurant.csv')

print (train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        12 non-null     int64 
 1   Alt       12 non-null     object
 2   Bar       12 non-null     object
 3   Fri       12 non-null     object
 4   Hun       12 non-null     object
 5   Pat       10 non-null     object
 6   Price     12 non-null     object
 7   Rain      12 non-null     object
 8   Res       12 non-null     object
 9   Type      12 non-null     object
 10  Est       12 non-null     object
 11  WillWait  12 non-null     object
dtypes: int64(1), object(11)
memory usage: 1.2+ KB
None


In [37]:
print(train.head(10))

   id  Alt  Bar  Fri  Hun   Pat Price Rain  Res     Type     Est WillWait
0   1  Yes   No   No  Yes  Some   $$$   No  Yes   French    0-10      Yes
1   2  Yes   No   No  Yes  Full     $   No   No     Thai   30-60       No
2   3   No  Yes   No   No  Some     $   No   No   Burger    0-10      Yes
3   4  Yes   No  Yes  Yes  Full     $  Yes   No     Thai   10-30      Yes
4   5  Yes   No  Yes   No  Full   $$$   No  Yes   French     >60       No
5   6   No  Yes   No  Yes  Some    $$  Yes  Yes  Italian    0-10      Yes
6   7   No  Yes   No   No   NaN     $  Yes   No   Burger    0-10       No
7   8   No   No   No  Yes  Some    $$  Yes  Yes     Thai    0-10      Yes
8   9   No  Yes  Yes   No  Full     $  Yes   No   Burger     >60       No
9  10  Yes  Yes  Yes  Yes  Full   $$$   No  Yes  Italian   10-30       No


In [38]:
nptest = np.array(train["WillWait"])
nptest.sort()
nptest

array(['No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes'], dtype=object)

In [44]:
YesNo_Mapping = {
    "Yes": 1,
    "No": 0,
}

train["Alt"] = train["Alt"].replace(YesNo_Mapping)
train["Bar"] = train["Bar"].replace(YesNo_Mapping)
train["Fri"] = train["Fri"].replace(YesNo_Mapping)
train["Hun"] = train["Hun"].replace(YesNo_Mapping)
train["Rain"] = train["Rain"].replace(YesNo_Mapping)
train["Res"] = train["Res"].replace(YesNo_Mapping)
train["WillWait"] = train["WillWait"].replace(YesNo_Mapping)

Pat_Mapping = {
    "None": 0,
    "Some": 0.5,
    "Full": 1,
}

train["Pat"] = train["Pat"].replace(Pat_Mapping)

Price_Mapping = {
    "$": 1,
    "$$": 2,
    "$$$": 3,
}
train["Price"] = train["Price"].replace(Price_Mapping)

Type_Mapping = {
    "French": 1,
    "Thai": 2,
    "Burger": 3,
    "Italian": 4,
}
train["Type"] = train["Type"].replace(Type_Mapping)

In [41]:
# print(train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean())
GroupedByType = train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean()

print(GroupedByType)

      Type  WillWait
0   Burger       0.5
1   French       0.5
2  Italian       0.5
3     Thai       0.5


In [42]:
GroupedByPat = train[["Pat", "WillWait"]].groupby(["Pat"], as_index=False).mean()

print(GroupedByPat)

    Pat  WillWait
0  Full  0.333333
1  Some  1.000000
